# Задание по программированию: Примеры задач анализа текстов
## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("`spam`") и не спам ("`ham`"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

## Задание

1) Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection).

`smsspamcollection.zip`

2) Считайте датасет в `Python` (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [1]:
import pandas as pd

dataset = pd.read_csv('SMSSpamCollection.txt', '\t', header=None, names=('class', 'text'))
dataset.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3) Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте `1` для спама и `0` для "не спама".

In [2]:
dataset['class'].replace(to_replace=['ham', 'spam'], value=[0, 1], inplace=True)
dataset.head()

,class,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


4) Используя `sklearn.feature_extraction.text.CountVectorizer` со стандартными настройками, получите из списка текстов матрицу признаков `X`.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vect = CountVectorizer()
X_train = vect.fit_transform(dataset['text'])

5) Оцените качество классификации текстов с помощью `LogisticRegression()` с параметрами по умолчанию, используя `sklearn.cross_validation.cross_val_score` и посчитав среднее арифметическое качества на отдельных `fold`'ах. Установите `random_state=2`. Параметр `cv` задайте равным `10`. В качестве метрики качества используйте `f1`-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до `1` знака после запятой.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

clf = LogisticRegression(random_state=2)
answer1 = cross_val_score(clf, X_train, dataset['class'], cv=10, scoring='f1').mean()
print round(answer1, 1)
with open("q5.txt", "w") as fout:
    fout.write(str(round(answer1, 1)))

C:\Users\Insassin\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.9


6) А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

    "FreeMsg: Txt: claim your reward of 3 hours talk time"

    "Have you visited the last lecture on physics?"

    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

    "Only 99$"

Прогнозы классификатора (`0` - не спам, `1` - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [5]:
X_test = vect.transform(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", 
          "FreeMsg: Txt: claim your reward of 3 hours talk time", "Have you visited the last lecture on physics?", 
          "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
          "Only 99$"])
clf.fit(X_train, dataset['class'])
answer2 = clf.predict(X_test)
print answer2
with open("q6.txt", "w") as fout:
    fout.write(" ".join([str(num) for num in answer2]))

[1 1 0 0 0]


7) Задайте в `CountVectorizer` параметр `ngram_range=(2,2)`, затем `ngram_range=(3,3)`, затем `ngram_range=(1,3)`. Во всех трех случаях измерьте получившееся в кросс-валидации значение `f1`-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки `n`-граммы для разных диапазонов `n` - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [6]:
def n_gramms(dims, clf):
    vect = CountVectorizer(ngram_range=dims)
    X_train = vect.fit_transform(dataset['text'])
    return round(cross_val_score(clf, X_train, dataset['class'], cv=10, scoring='f1').mean(), 2)

answer3 = []
answer3.append(n_gramms(dims=(2,2), clf=clf))
answer3.append(n_gramms(dims=(3,3), clf=clf))
answer3.append(n_gramms(dims=(1,3), clf=clf))
print answer3
with open("q7.txt", "w") as fout:
    fout.write(" ".join([str(num) for num in answer3]))

[0.82, 0.73, 0.93]


8) Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии `MultinomialNB()`. Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через `Pipeline` происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [7]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
answer4 = []
answer4.append(n_gramms(dims=(2,2), clf=clf))
answer4.append(n_gramms(dims=(3,3), clf=clf))
answer4.append(n_gramms(dims=(1,3), clf=clf))
print answer4
with open("q8.txt", "w") as fout:
    fout.write(" ".join([str(num) for num in answer4]))

[0.65, 0.38, 0.89]


9) Попробуйте использовать в логистической регрессии в качестве признаков `Tf*idf` из `TfidfVectorizer` на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с `CountVectorizer` на униграммах? (напишите в файле с ответом `1`, если повысилось, `-1`, если понизилось, и `0`, если изменилось не более чем на `0.01`). Обратите внимание, что результат перехода к `tf*idf` не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [8]:
vect = TfidfVectorizer()
X_train = vect.fit_transform(dataset['text'])

clf = LogisticRegression(random_state=2)
answer5 = cross_val_score(clf, X_train, dataset['class'], cv=10, scoring='f1').mean()
print 'CountVectorizer:', answer1, '\nTfidfVectorizer:', answer5
with open("q9.txt", "w") as fout:
    if abs(answer5-answer1) < 0.01:
        fout.write(str(0))
    elif answer5 > answer1:
        fout.write(str(1))
    else:
        fout.write(str(-1))

CountVectorizer: 0.932640298361 
TfidfVectorizer: 0.852859955417
